In [129]:
import numpy as np
import math

In [130]:
# интегрируемая функция
def f(x):
    return 1/(1 + np.sin(x) + np.power(x, 2))

# промежуток интегрирования
a, b = 0, math.pi / 4

# точность
eps = 10**(-4)

In [136]:
# коэффициенты для правила Рунге
theta_vals = {'right_rect' : 1/3, 'trapezium' : 1/3, 
            'simps' : 1/15}

# коэффициенты для формулы Гаусса
gaus_x = {2: [-0.55773502692, 0.55773502692],
          3: [-0.7745966692, 0, 0.7745966692],
          4: [-0.8611363116, -0.3399810436, 0.3399810436, 0.8611363116]}
gaus_a = {2: [1, 1],
          3: [5/9, 8/9, 5/9],
          4: [0.3478548451, 0.6521451549, 0.6521451549, 0.3478548451]}

# метод правых прямоугольников
def right_rect(n):
    points = np.linspace(a, b, n+1)
    h = points[1:] - points[:-1]
    return np.sum(f(points[1:]) * h)

# метод трапеция
def trapezium(n):
    points = np.linspace(a, b, n+1)
    h = points[1:] - points[:-1]
    return np.sum((f(points[:-1]) + f(points[1:])) / 2 * h)

# метод Симпсона
def simps(n):
    points = np.linspace(a, b, n+1)
    h = points[1:] - points[:-1]
    return np.sum((f(points[:-1]) + f(points[1:]) + 
            4 * f((points[:-1] + points[1:])/2)) * h / 6)

# функция для замены в формуле Гаусса
def scale(x):
    return (a + b) / 2 + (b - a) * x / 2 

# формула Гаусса
def gaus(n):
    assert(2 <= n <= 4)
    result = 0
    for (xi, ai) in zip(gaus_x[n],gaus_a[n]):
        result += ai * f(scale(xi))
    return result * (b-a)/2

# оценивает погрешность для шага 2n
# для данного θ по правилу Рунге 
def runge(prev, curr, theta):
    return np.abs(curr - prev) * theta

# выводит в консоль строку таблицы в формате LaTeX
def print_table_line(a, b, c):
    print(f'& {a} & {b} & {c} \\\\ \\hline')

# выводит в консоль таблицу для данного метода
# в формате LaTeX
def show_latex_table(method, method_name):
    theta = theta_vals[method_name]
    n = 1
    h = b - a
    curr = method(n)
    error = math.inf
    print(f'& $h = {h:.6f}$ & $I_h={curr:.6f}$ &  \\\\ \\hline')
    while error > eps:
        n *= 2
        h /= 2
        prev, curr = curr, method(n)
        error = runge(prev, curr, theta)
        print(f'& $h/{n} = {h:.6f}$ & $I_{{h/{n}}}={curr:.6f}$ & $R_{{h/{n}}}=\
{error:.6f}$ \\\\ \\hline')

In [132]:
# выведем таблицу для метода правых прямоугольников
show_latex_table(right_rect, 'right_rect')

& $h = 0.785398$ & $I_h=0.337957$ &  \\ \hline
& $h/2 = 0.392699$ & $I_{h/2}=0.424493$ & $R_{h/2}=0.028845$ \\ \hline
& $h/4 = 0.196350$ & $I_{h/4}=0.474612$ & $R_{h/4}=0.016706$ \\ \hline
& $h/8 = 0.098175$ & $I_{h/8}=0.501169$ & $R_{h/8}=0.008852$ \\ \hline
& $h/16 = 0.049087$ & $I_{h/16}=0.514802$ & $R_{h/16}=0.004544$ \\ \hline
& $h/32 = 0.024544$ & $I_{h/32}=0.521706$ & $R_{h/32}=0.002301$ \\ \hline
& $h/64 = 0.012272$ & $I_{h/64}=0.525180$ & $R_{h/64}=0.001158$ \\ \hline
& $h/128 = 0.006136$ & $I_{h/128}=0.526922$ & $R_{h/128}=0.000581$ \\ \hline
& $h/256 = 0.003068$ & $I_{h/256}=0.527795$ & $R_{h/256}=0.000291$ \\ \hline
& $h/512 = 0.001534$ & $I_{h/512}=0.528232$ & $R_{h/512}=0.000146$ \\ \hline
& $h/1024 = 0.000767$ & $I_{h/1024}=0.528450$ & $R_{h/1024}=0.000073$ \\ \hline


In [133]:
# выведем таблицу для метода трапеций
show_latex_table(trapezium, 'trapezium')

& $h = 0.785398$ & $I_h=0.561678$ &  \\ \hline
& $h/2 = 0.392699$ & $I_{h/2}=0.536353$ & $R_{h/2}=0.008441$ \\ \hline
& $h/4 = 0.196350$ & $I_{h/4}=0.530542$ & $R_{h/4}=0.001937$ \\ \hline
& $h/8 = 0.098175$ & $I_{h/8}=0.529134$ & $R_{h/8}=0.000470$ \\ \hline
& $h/16 = 0.049087$ & $I_{h/16}=0.528785$ & $R_{h/16}=0.000116$ \\ \hline
& $h/32 = 0.024544$ & $I_{h/32}=0.528697$ & $R_{h/32}=0.000029$ \\ \hline


In [134]:
# выведем таблицу для метода Симпсона
show_latex_table(simps, 'simps')

& $h = 0.785398$ & $I_h=0.527912$ &  \\ \hline
& $h/2 = 0.392699$ & $I_{h/2}=0.528605$ & $R_{h/2}=0.000046$ \\ \hline


In [135]:
# выведем приближенные значения интеграла,
# подсчитанные по формуле Гаусса
for i in range(2, 5):
    print(f'{i}:\t{gaus(i):.10f}')

2:	0.5279828144
3:	0.5286849071
4:	0.5286680863
